In [1]:
PYTHON_PATHS = [".."]
import sys
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)

In [2]:
import numpy as np
import os
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import kapture
import cv2
import tempfile

In [3]:
from unsupkeypoints.data import DescriptorDataModule
from unsupkeypoints.models import PointNetwork, BinarizationPointNetwork, SimpleModel
from unsupkeypoints.criterions import RegressionLoss, RGBandModelReprojectionLoss
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.parsing import AttributeDict
import torch
import pytorch_lightning as pl
import tqdm
from collections import Counter

In [4]:
from clearml import Task

In [5]:
from unsupkeypoints.utils.universal_factory import UniversalFactory

In [6]:
factory = UniversalFactory([PointNetwork, RegressionLoss, RGBandModelReprojectionLoss,
                            BinarizationPointNetwork, SimpleModel])

In [7]:
task = Task.init(project_name="unsup-3d-keypoints", task_name="3d position from descriptor")

ClearML Task: created new task id=b0c6e2d22fbf42fcac7073d2d0226662
ClearML results page: https://app.community.clear.ml/projects/f773f19fb1d94708840bc5460e97ba2b/experiments/b0c6e2d22fbf42fcac7073d2d0226662/output/log


# Data module

In [8]:
from clearml import Dataset
dataset_path = Dataset.get(dataset_id="40300e50677746148777e4a3c3a21ba9").get_local_copy()
# dataset_path = "/home/mikhail/.clearml/cache/storage_manager/datasets/ds_40300e50677746148777e4a3c3a21ba9"

In [9]:
TRAIN_PATH = os.path.join(dataset_path, "output_kapture/7scenes/fire/mapping")
TEST_PATH = os.path.join(dataset_path, "output_kapture/7scenes/fire/query")

In [10]:
data_module = DescriptorDataModule(TRAIN_PATH, TEST_PATH)

[DescriptorDataModule] - train dataset size 389950
[DescriptorDataModule] - test dataset size 346052


# Model

In [11]:
params = AttributeDict(
    name="BinarizationPointNetwork",
    optimizer=AttributeDict(
        lr=1e-3
    ),
    classifier=AttributeDict(
        name="SimpleModel",
        input_dimension=512,
        hidden_dimensions=(256,),
        output_dimension=64
    ),
    regressor=AttributeDict(
        name="SimpleModel",
        input_dimension=64,
        hidden_dimensions=(64,),
        output_dimension=3
    ),
    criterion={
        "name": "RGBandModelReprojectionLoss",
        "minimal_depth": 0.1,
        "maximal_distance": 0.05,
        "maximal_reprojection_loss": 1000,
        "robust_maximal_reprojection_loss": 100,
        "distance_coef": 1000
    },
    probabilistic=True,
    sigmoid_after_classifier=True,
    max_probability_error=0.45,
    metric_logging_frequency=2,
)
task.connect(params)
model = factory.make_from_parameters(params)

In [12]:
trainer_params = {
    "max_epochs": 10,
    "gpus": 1,
    "batch_size": 512,
    "checkpoint_every_n_val_epochs": 2,
    "check_val_every_n_epoch": 2
}
task.connect(trainer_params)
# model_checkpoint_directory = tempfile.mkdtemp(dir=os.path.dirname(task.cache_dir))
model_checkpoint = pl.callbacks.ModelCheckpoint(
    every_n_val_epochs=trainer_params["checkpoint_every_n_val_epochs"])
data_module._batch_size = trainer_params["batch_size"]
trainer = factory.kwargs_function(pl.Trainer)(
    logger=TensorBoardLogger(os.path.join(os.path.dirname(task.cache_dir), "lightning_logs"), name="fire"),
    callbacks=[model_checkpoint],
    **trainer_params)

INFO    ::pytorch_lightning.utilities.distributed: GPU available: True, used: True
INFO    ::pytorch_lightning.utilities.distributed: TPU available: False, using: 0 TPU cores


In [13]:
trainer.fit(model, data_module)

INFO    ::pytorch_lightning.accelerators.gpu: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO    ::pytorch_lightning.core.lightning: 
  | Name        | Type                        | Params
------------------------------------------------------------
0 | _classifier | SimpleModel                 | 147 K 
1 | _regressor  | SimpleModel                 | 4.4 K 
2 | _criterion  | RGBandModelReprojectionLoss | 0     
------------------------------------------------------------
152 K     Trainable params
0         Non-trainable params
152 K     Total params
0.609     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:610: LightningDeprecationWarning:

Relying on `self.log('val_loss', ...)` to set the ModelCheckpoint monitor is deprecated in v1.2 and will be removed in v1.4. Please, create your own `mc = ModelCheckpoint(monitor='your_monitor')` and use it as `Trainer(callbacks=[mc])`.

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:897: UserWarning:

Detected KeyboardInterrupt, attempting graceful shutdown...



In [14]:
trainer.test(model, data_module.test_dataloader())

INFO    ::pytorch_lightning.accelerators.gpu: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_distance_loss': 0.2339845895767212,
 'test_good_point_ratio': 0.07517945021390915,
 'test_good_reprojection_loss': nan,
 'test_loss': 232.11334228515625,
 'test_median_position_error': 0.6230296025998491,
 'test_median_rotation_error': 16.12280463762646,
 'test_point_count': 2000.0,
 'test_reprojection_loss': 115.14398956298828}
--------------------------------------------------------------------------------


[{'test_loss': 232.11334228515625,
  'test_reprojection_loss': 115.14398956298828,
  'test_distance_loss': 0.2339845895767212,
  'test_good_reprojection_loss': nan,
  'test_good_point_ratio': 0.07517945021390915,
  'test_median_position_error': 0.6230296025998491,
  'test_median_rotation_error': 16.12280463762646,
  'test_point_count': 2000.0}]

In [15]:
task.close()

2021-07-30 14:09:56,766 - clearml.Task - INFO - Waiting to finish uploads
2021-07-30 14:09:57,810 - clearml.Task - INFO - Finished uploading


KeyboardInterrupt: 